<a href="https://colab.research.google.com/github/MSFTserver/AI-Colab-Notebooks/blob/main/NNST_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Batch] Neural Neighbor Style Transfer 
https://github.com/nkolkin13/NeuralNeighborStyleTransfer
<br><br>

Colab ~~Repo~~ for the algorithm NNST-Opt, described in the preprint "Neural Neighbor Style Transfer", please feel free to email any questions to kolkin@adobe.com Paper Link: https://ttic.uchicago.edu/~nickkolkin/Paper/NNST_Preprint.pdf

----
<br><br>
made to run in colab by [MSFTserver](https://gist.github.com/MSFTserver) aka [HostsServer](https://twitter.com/hostsserver)

In [ ]:
#@markdown # 1) Download & Import Dependecies

# Core Imports
import time, argparse, random, sys, os
from google.colab import drive

# External Dependency Imports
from imageio import imwrite
import torch
import numpy as np
from IPython.display import Image

# Internal Project Imports
!git clone https://github.com/nkolkin13/NeuralNeighborStyleTransfer
sys.path.append('/content/NeuralNeighborStyleTransfer')
from pretrained.vgg import Vgg16Pretrained
from utils import misc as misc
from utils.misc import load_path_for_pytorch
from utils.stylize import produce_stylization

In [ ]:
#@markdown # 2) Prepare Folders and Mount Drive
#@markdown ### <font color="red"> READ BEFORE RUNNING </font>

def createPath(filepath):
    if os.path.exists(filepath) == False:
      os.makedirs(filepath)
      print(f'Made {filepath}')
    else:
      print(f'filepath {filepath} exists.')

#@markdown specify the folder the images are located in your google drive

#@markdown this will also be used as the path to create the `out` folder
image_folder = "AI/NNST" #@param {type:"string"}

drive.mount('/content/drive')
root_path = f'/content/drive/MyDrive/{image_folder}'

input_path = f'{root_path}'
output_path = f'{root_path}/out'

createPath(output_path)

In [ ]:
#@markdown # 4) Upload Style Image
#@markdown run this code block to see the `browse` dialog box to upload files in the output window
import os
from google.colab import files
import shutil

uploaded_style_image = files.upload()
style_image_file_name = list(uploaded_style_image.keys())[0]
style_path = os.path.join('/content', style_image_file_name)

from IPython.display import Image
Image(style_path, height=500)

In [ ]:
#@markdown # 5) Config
# Fix Random Seed
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# Define Configuration
high_res = False #@param {type:"boolean"}
cpu = False #@param {type:"boolean"}
no_flip = False #@param {type:"boolean"}
content_loss = False #@param {type:"boolean"}
dont_colorize = False #@param {type:"boolean"}
alpha=0.75 #@param {type:"number"}

# Interpret config options arguments
max_scls = 4
sz = 512
if high_res:
    max_scls = 5
    sz = 1024
flip_aug = (not no_flip)
misc.USE_GPU = (not cpu)
content_weight = 1. - alpha

# Error checking for arguments
# error checking for paths deferred to imageio
assert (0.0 <= content_weight) and (content_weight <= 1.0), "alpha must be between 0 and 1"
assert torch.cuda.is_available() or (not misc.USE_GPU), "attempted to use gpu when unavailable"

In [ ]:
import base64
#@markdown # 6) Run Style Transfer
def doRun(input_path,style_path,file_number,image_name):
  # Define feature extractor
  cnn = misc.to_device(Vgg16Pretrained())
  phi = lambda x, y, z: cnn.forward(x, inds=y, concat=z)

  # Load images
  content_im_orig = misc.to_device(load_path_for_pytorch(input_path, target_size=sz)).unsqueeze(0)
  style_im_orig = misc.to_device(load_path_for_pytorch(style_path, target_size=sz)).unsqueeze(0)

  # Run Style Transfer
  torch.cuda.synchronize()
  start_time = time.time()
  output = produce_stylization(content_im_orig, style_im_orig, phi,
                              max_iter=200,
                              lr=2e-3,
                              content_weight=content_weight,
                              max_scls=max_scls,
                              flip_aug=flip_aug,
                              content_loss=content_loss,
                              dont_colorize=dont_colorize)
  torch.cuda.synchronize()
  print('Done! image time: {}'.format(time.time() - start_time))

  # Convert from pyTorch to numpy, clip to valid range
  new_im_out = np.clip(output[0].permute(1, 2, 0).detach().cpu().numpy(), 0., 1.)

  # Save stylized output
  save_im = (new_im_out * 255).astype(np.uint8)
  print()
  imwrite(f'{output_path}/{image_name}-styled{image_number}.jpg', save_im)
  Image(f'{output_path}/{image_name}-styled{image_number}.jpg')
  # Free gpu memory in case something else needs it later
  if misc.USE_GPU:
      torch.cuda.empty_cache()

image_number = 0
batch_start_time = time.time()
for image_file in os.scandir(input_path):
    if image_file.is_file():
      image_number = image_number + 1
      image_name, image_ext = os.path.splitext(os.path.basename(image_file.path))
      new_input_path = os.path.join(input_path, f'{image_name}{image_ext}')
      print(f'Processing Image #{image_number} {image_name}{image_ext}')
      doRun(new_input_path,style_path,image_number,image_name)
print(f'Done! batch of {image_number} images in: {time.time() - batch_start_time}')